# Introdução

Neste documento faremos o preprocessamento dos textos para criar os casos testes e salvá-los. Dessa forma, poderemos retestar sem aleatoriedade de eventos.

In [42]:
# !pip install langchain==0.0.340 langchain --force-reinstall

In [43]:
from datetime import date

In [44]:
validation_tests_partition = str(date.today())

In [45]:
validation_tests_path = f'./validation_tests/{validation_tests_partition}'
validation_tests_path

'./validation_tests/2024-06-09'

# 1. Recuperação dos Documentos Formatados

In [46]:
import os
import json

In [47]:
pdf_folder_path = "./formatted_documents"

In [48]:
filename_list = []
for filename in os.listdir(pdf_folder_path):
    if filename.endswith(".json"):
        filename_list.append(os.path.join(pdf_folder_path, filename))

filename_list = sorted(filename_list)

In [49]:
len(filename_list)

1538

In [50]:
def retrieve_document(filename):
    with open(filename, "r") as file:
        json_file = json.load(file)
    return json_file

## Teste

In [51]:
json_document = retrieve_document(filename_list[10])

In [52]:
json_document.keys()

dict_keys(['title', 'authors', 'abstract', 'bibliography_pages', 'keywords', 'urls', 'pdf_url', 'id', 'sentences'])

In [53]:
json_document['title']

'O lugar dos homens na natureza social : uma proposta reflexiva sobre o meio ambiente'

In [54]:
json_document['authors']

['Bessa, Eliane da Silva.',
 'Randolph, Rainer orient.',
 'Universidade Federal do Rio de Janeiro. Instituto de Pesquisa e Planejamento Urbano e Regional.']

In [55]:
json_document['abstract'][:300] + '...'

'Focaliza o meio ambiente como objeto de estudo. Parte da sua problematização como questão social para chegar à definição de uma proposta teórico-metodológica, cujo objetivo é entender o meio ambiente como forma particular de organização do território. O percurso que vincula um ponto ao outro passa, ...'

In [56]:
json_document['keywords']

['Meio ambiente.', 'Conflito social.', 'Organização territorial.']

In [57]:
json_document['urls']

['http://objdig.ufrj.br/42/teses/81683.pdf']

In [58]:
print(len(json_document["sentences"]))

0


# 2. Seleção dos Testes

**Escolha dos Testes:**

1. Utilizaremos uma fração de 1000 abstracts dos artigos como input original.
2. Obteremos os 4 abstracts mais similares a cada um destes via embedding.
3. Para cada abstract original, utilizaremos uma LLM para sumarizar o tópico/tema de pesquisa.

### Reescrita com LLM

In [59]:
from langchain.schema import SystemMessage
from langchain.chat_models import ChatOpenAI

In [19]:
openai_api_key = "xxxxxxx"

In [73]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature=1,
    # temperature=0.2,
    openai_api_key=openai_api_key,       
    max_tokens=4096
)

In [74]:
# def get_research_topic(title, abstract):
#     query = f"""
#         Dado o seguinte título e resumo de um artigo científico, sua tarefa é retornar o tópico central da pesquisa.
        
#         [TITULO] {title}
#         [RESUMO] {abstract}
        
#         Qual o tópico central da pesquisa? Ou seja, o que busca-se pesquisar?
#         Não retorne descobertas, conclusões ou qualquer tarefa que tenha sido realizado pela pesquisa, apenas o tópico de pesquisa.
#         Não inicie com "O tópico central da pesquisa é...", vá direto ao ponto.
#         Responda com no mínimo 40 palavras e no máximo 80 palavras.
#     """
    
#     messages = [SystemMessage(content=query)]
#     response = llm(messages).content
#     return response

In [75]:
# def get_research_topic(title, abstract):
#     query = f"""
#         Você é um pesquisador que está buscando realizar uma revisão de literatura sobre a nova pesquisa que está desenvolvendo. Esse é um resumo da nova pesquisa:
#         "{abstract}"
        
#         Instruções:
#         Identifique qual o objeto de pesquisa, ou seja, seu objetivo de estudo, e resuma.
#         Não responda conclusões, descobertas ou o que foi feito na pesquisa, apenas é do interesse o tópico de pesquisa.
#         Não inicie com "O tópico central da pesquisa é...", vá direto ao ponto.
#         Retorne o resumo sobre o tópico de pesquisa com no mínimo 40 palavras e no máximo 80 palavras:
#     """
    
#     messages = [SystemMessage(content=query)]
#     response = llm(messages).content
#     return response

In [78]:
def get_research_topic(title, abstract):
#     query = f"""
#         Você é um pesquisador que está buscando realizar uma revisão de literatura sobre a nova pesquisa que está desenvolvendo. Esse é um resumo da nova pesquisa:
#         "{abstract}"
        
#         Instruções:
#         Identifique qual o objetivo da pesquisa e resuma.
#         Atenção, não é relevante conclusões, descobertas ou dizer o que foi realizado, queremos apenas o objetivo inicial da pesquisa.
#         Retorne o resumo do objetivo com mínimo de 40 palavras e máximo 80 palavras:
#     """

    # teste do joao
    query = f"""
        Você é um pesquisador que está buscando realizar uma revisão de literatura sobre a nova pesquisa que está desenvolvendo. Esse é um resumo da nova pesquisa:
        "{abstract}"
        
        Instruções:
        Qual o objetivo inicial da pesquisa? Resuma com mínimo de 40 palavras e máximo 80 palavras:
    """
    
    messages = [SystemMessage(content=query)]
    response = llm(messages).content
    return response

In [79]:
# Test
json_document = retrieve_document(filename_list[120])

query = get_research_topic(json_document['title'], json_document['abstract'])

print("Title:", json_document['title'], "\n")
print("Abstract:", json_document['abstract'], "\n")
print("Query:", query, "\n")

Title: Acolhimento do Centro de Doença de Alzheimer do IPUB–UFRJ para idosos e seus cuidadores 

Abstract:  Centro de Doença de Alzheimer do IPUB é referência no atendimento de idosos com demência, no Rio de Janeiro. A enfermeira e a assistente social do setor criaram conjuntamente o serviço de acolhimento, para avaliação de novos usuários. Objetivo geral: Descrever a intervenção da enfermeira no acolhimento do CDA. Objetivos específicos: Apontar demandas em atenção psicossocial e psicogeriátrica dos usuários; Produzir materiais que inspirem possibilidades de atenção psicossocial a idosos em outros serviços de saúde mental. Como produto de contribuição técnica foi elaborado o fluxograma do serviço de acolhimento do CDA. A metodologia adotada foi o estudo de caso descritivo. Trata-se de uma pesquisa qualitativa, utilizando-se o instrumento de anamnese do serviço e um condensado de observações do campo de prática como fonte de dados. Em seu bojo são apresentados o percurso histórico do d

### Artigos Similares

In [25]:
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings

print("Recuperando Embeddings...")
embeddings = SentenceTransformerEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

print("Construindo Vector Database...")
vectorstore = FAISS.load_local("./faiss_vector_store/", embeddings)

Recuperando Embeddings...
Construindo Vector Database...


In [26]:
import cohere

cohere_key = "xxxxx"
cohere_client = cohere.Client(cohere_key)

In [27]:
def get_similar_abstracts(abstract, original_id, k=5, fetch_k=10000):
    res = vectorstore.similarity_search_with_score(
        abstract,
        k=k*2,
        fetch_k=fetch_k*2,
        filter={"type":"abstract"}
    )
    
    res = list(set([r[0].metadata['id'] for r in res if (r[0].metadata['id'] != original_id and r[1] < 1.5)]))

    # if len(res) == 0:
    #     res_reranking = []
    # else:
    #     reranking_response = cohere_client.rerank(
    #       model = "rerank-multilingual-v2.0",
    #       query = abstract,
    #       documents = res,
    #       top_n = k-1
    #     )
    #     res_reranking = [res[r.index] for r in reranking_response if r.relevance_score > 0.5]

    return res[:k-1] #, res_reranking

In [28]:
# test
# get_similar_abstracts(json_document['abstract'], json_document['id'], k=5)

### Execução

In [80]:
import random
random.seed(42)

In [84]:
sample_filename_list = random.sample(filename_list, 100)
# sample_filename_list = random.sample(filename_list, 10)       # test sample!!!

In [85]:
len(sample_filename_list)

100

In [86]:
test_info = []

for filename in sample_filename_list:
    doc = retrieve_document(filename).copy()
    
    query = get_research_topic(doc['title'], doc['abstract'])
    # similar = get_similar_abstracts(doc['abstract'], doc['id'], k=5)
   
    test_info.append({
        "id": doc["id"],
        "title": doc["title"],
        "abstract": doc["abstract"],
        "query": query,
        # "similar": similar,
        # "similar_rerank": similar_rerank
    })
    
    print('.', end='')
    if len(test_info) % 100 == 0: print(len(test_info))

....................................................................................................100


In [37]:
# for 1000
# started 22:44
# ended 23:27

In [38]:
# for info in test_info:
#     print("title:", info['title'])
#     print("query:", info["query"])
#     print("similar:", info["similar"])
    # print("similar rerank:", info["similar_rerank"])
    # print()

# 3. Salvamento dos Testes

In [87]:
import json

In [88]:
import os
os.mkdir(validation_tests_path)
print(validation_tests_path)

./validation_tests/2024-06-09


In [89]:
with open(f'{validation_tests_path}/test_info.json', 'w') as fp:
    json.dump(test_info, fp)